<a href="https://colab.research.google.com/github/paella-e/butterfly/blob/main/capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 00. Butterfly Classification Notebook Overview

This notebook builds a butterfly classification model. There are 75 different classifications.

**Data**: [Butterfly Image Classification](https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification) dataset from Kaggle.

* There are 6,499 labeled images in the training set.
* There are 2,786 unlabeled images in the test set.


## 01. Getting the Notebook ready: Utilize GPU & Import necessary libraries

This notebook uses TensorFlow libraries to train our model. This requires imports. Additionally, Machine Learning applications benefit from running on a GPU rather than CPU, so we will want to change our Runtime environment.

In [4]:
# Change Runtime from CPU > GPU
# In Google Colab, go to Runtime > Change runtime type > GPU
# This notebook was made with the T4 GPU option

# Import TensorFlow and TensorFlow Hub
import tensorflow as tf
import tensorflow_hub as hub

# Confirm we have succesfully changed our untime to use a GPU.
print("Success: Using GPU") if tf.config.list_physical_devices() else "Warning: not using GPU"

Success: Using GPU


## 02. Getting the data ready to use in Google Colab

1. Download the [Butterfly Image Classificaiton](https://www.kaggle.com/competitions/cloud-recognition/data) dataset from Kaggle.
2. Upload the zip file to Google Drive
  * If you cannot use Google Drive, you can upload the data directly to the session, however it will expire with the session and need to be re-uploaded in every subsequent session.
3. Unzip it with the following command:
> !unzip "*ZIP FILE LOCATION*" -d "*DESTINATION PATH*"



In [15]:
# Remove the hashtag symbol at the start of the line below and then re-run the cell to run the unzip command if needed
#!unzip "/content/drive/MyDrive/Butterfly/archive.zip" -d "/content/drive/MyDrive/Butterfly/"